In [24]:
# pip install transformers==4.40.1 datasets==2.19.0 sentence-transformers==2.7.0 faiss-cpu==1.8.0 llama-index==0.10.34 llama-index-embeddings-huggingface==0.2.0 -qqq

## 예제 10.1 문장 임베딩을 활용한 단어 간 유사도 계산

In [ ]:
# 문장을 벡터로 변환해주는 사전학습된 모델을 로딩/사용
from sentence_transformers import SentenceTransformer 

# 두 벡터 간의 코사인 유사도를 계산하는 함수
# 코사인 유사도는 두 벡터 사이의 각도를 기반으로 유사도를 측정
# 코사인 유사도 범위 : -1 ~ 1 (1 매우 유사, 0 관련 없음, -1 정반대)
from sklearn.metrics.pairwise import cosine_similarity


# snunlp/KR-SBERT-V40K-klueNLI-augSTS : 한국어 전용 SBERT(Sentence-BERT) 모델
smodel = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

dense_embeddings = smodel.encode(['학교', '공부', '운동'])
cosine_similarity(dense_embeddings)[0] # 학교 기준


#                  학교          공부        운동
# array([학교 : [0.99999994, 0.5950743 , 0.32537544],
#        공부 : [0.5950743 , 1.        , 0.54595673],
#        운동 : [0.32537544, 0.54595673, 1.0000002 ]], dtype=float32)

array([0.99999994, 0.5950743 , 0.32537544], dtype=float32)

## 예제 10.2 원핫 인코딩의 한계

In [48]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 원핫 인코딩된 벡터를 딕셔너리로 정의
word_dict = {
    "school": np.array([[1, 0, 0]]),
    "study": np.array([[0, 1, 0]]),
    "workout": np.array([[0, 0, 1]])
}

# school과 study 사이의 코사인 유사도 계산
# 두 벡터는 완전히 직각(orthogonal)이므로 유사도는 0
cosine_school_study = cosine_similarity(word_dict["school"], word_dict['study'])  # 0.0

# school과 workout 사이의 코사인 유사도 계산
# 마찬가지로 전혀 관련 없는 차원 => 유사도는 0
cosine_school_workout = cosine_similarity(word_dict['school'], word_dict['workout'])  # 0.0

print("Cosine similarity (school, study):", cosine_school_study[0][0])
print("Cosine similarity (school, workout):", cosine_school_workout[0][0])


Cosine similarity (school, study): 0.0
Cosine similarity (school, workout): 0.0


In [ ]:
word_embeddings = {
    "school": np.array([[0.9, 0.8, 0.1]]),
    "study":  np.array([[0.85, 0.75, 0.2]]),
    "workout": np.array([[0.1, 0.2, 0.95]])
}

# 임베딩 기반 유사도 측정
cosine_school_study_embed = cosine_similarity(word_embeddings["school"], word_embeddings["study"])  
cosine_school_workout_embed = cosine_similarity(word_embeddings["school"], word_embeddings["workout"])

print("\n[Using Embedding Vectors]")
print("Cosine similarity (school, study):", cosine_school_study_embed[0][0])      # e.g. ~0.99
print("Cosine similarity (school, workout):", cosine_school_workout_embed[0][0])  # e.g. ~0.4


[Using Embedding Vectors]
Cosine similarity (school, study): 0.9957846018984899
Cosine similarity (school, workout): 0.2925567851696626


## 예제 10.2 Sentence-Transformers 라이브러리로 바이 인코더 생성하기

In [60]:
from sentence_transformers import SentenceTransformer, models
# SentenceTransformer: Transformer + Pooling 등을 조합해 SBERT 모델을 구성하고,
# 문장을 임베딩하는 클래스.

# 1. 사용할 BERT 모델 로드
# 이 모델은 입력 문장을 BERT 방식으로 토큰화한 후, 각 단어/토큰에 대해 벡터를 생성함
word_embedding_model = models.Transformer('klue/roberta-base')

# 2. 풀링층 정의
# 문장 전체를 하나의 고정된 벡터로 만드는 풀링(Pooling) 레이어
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

# 3. 위 두 모듈(Transformer + Pooling)을 연결해 SentenceTransformer 모델로 구성
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

print(model.encode(['진정한 데이터 사이언스가 되는 길']).shape)
print(model.encode(['진정한 데이터 사이언스가 되는 길']))

c:\Users\me\anaconda3\envs\lang310\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(1, 768)
[[ 4.52818088e-02 -3.72968256e-01  9.29080248e-02 -6.31986186e-02
  -2.63903290e-01 -4.54857387e-02 -3.42952907e-01 -3.42025250e-01
  -3.49420041e-01  2.75499463e-01 -3.32062133e-02 -1.40977010e-01
   1.65380642e-01  3.45746964e-01  2.02794205e-02  6.19507954e-02
   2.59422548e-02 -1.37198254e-01 -2.29651388e-02 -1.13240995e-01
  -4.67441201e-01  1.21111870e-01  1.46921694e-01  4.28803802e-01
  -2.53313899e-01 -4.21574473e-01 -1.64629921e-01 -6.55333325e-02
  -2.41080523e-01 -2.53970772e-01 -1.90059230e-01 -2.43895695e-01
  -1.38974324e-01  1.42828703e-01  6.01070106e-01  8.84372219e-02
   3.58265527e-02  4.91743572e-02 -3.69422808e-02 -3.55697125e-01
  -6.39413148e-02 -1.04777902e-01 -2.05031052e-01 -9.72695351e-02
   3.87814522e-01 -6.81074038e-02 -2.31856316e-01  2.53421426e-01
  -2.98822850e-01 -2.59820670e-01  2.99505025e-01 -3.51672858e-01
   2.00901166e-01 -5.22441626e-01 -2.16984063e-01  5.24030440e-02
  -6.53395951e-02  2.38249362e-01 -2.17821136e-01 -1.39164314e-01
 

## 예제 10.2 코드로 살펴보는 평균 모드

In [ ]:

import torch
def mean_pooling(model_output, attention_mask):

    # model_output[0]: BERT의 마지막 은닉층 출력 (batch_size, seq_len, hidden_size)
    token_embeddings = model_output[0]

    # attention_mask: 실제 토큰인지(1) 패딩인지(0)를 나타냄
    # unsqueeze(-1): (batch_size, seq_len) → (batch_size, seq_len, 1)
    # expand(): 토큰 임베딩과 크기 맞춤 (broadcasting)
    # float(): 곱셈이 가능하도록 정수형 → 실수형으로 변환
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()

    # 마스크된 토큰 임베딩만 남기고 모두 더함 (sum along seq_len axis)
    # shape: (batch_size, hidden_size)
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)

    # 마스크 합을 통해 평균 낼 때 나눌 분모 계산 (0으로 나누는 걸 방지하기 위해 최소값 지정)
    sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)

    # 평균 벡터 구하기: (각 문장의 토큰 벡터 합 / 유효 토큰 수)
    # shape: (batch_size, hidden_size)
    return sum_embeddings / sum_mask


In [ ]:
# 1. transformer 모델에 문장을 입력하여 단어별 벡터 반환
# 2. 반환된 벡터를 평균내어 하나의 문장 벡터를 만듬

import torch
from transformers import AutoTokenizer, AutoModel

# 1. 모델과 토크나이저 로드
model_name = "klue/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 2. Mean Pooling 함수 정의
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] 
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)
    sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
    return sum_embeddings / sum_mask

# 3. 입력 문장
sentence = "학교에 갑니다"

# 4. 토크나이징 + 텐서 형태로 변환
inputs = tokenizer(sentence, return_tensors="pt")

# 5. 모델 추론 (gradient 계산 없이)
with torch.no_grad():
    model_output = model(**inputs)

# 6. Mean Pooling으로 문장 임베딩 추출
sentence_embedding = mean_pooling(model_output, inputs['attention_mask'])

# 7. 결과 출력
print("문장 임베딩 벡터:", sentence_embedding)
print("벡터 차원:", sentence_embedding.shape)


c:\Users\me\anaconda3\envs\lang310\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


문장 임베딩 벡터: tensor([[ 3.5191e-01, -4.1031e-01,  1.1834e-01,  5.3934e-02, -1.6616e-02,
         -1.1009e-01, -3.7938e-02, -2.7568e-01,  1.3821e-02, -4.5810e-02,
         -5.2823e-02,  9.9165e-02, -1.7758e-01,  4.5876e-02,  1.9208e-01,
         -1.5788e-01,  7.2701e-02, -3.7021e-01, -1.8652e-01, -5.7619e-01,
          2.0276e-01, -1.8577e-01,  7.0466e-02,  2.7253e-01, -1.3598e-01,
         -2.3917e-02,  2.5444e-01, -3.4262e-02,  3.2451e-02, -2.5701e-01,
         -6.7598e-01, -1.7985e-01,  2.0873e-01,  3.8534e-02,  5.8911e-01,
          1.5793e-01,  2.2785e-01,  2.8802e-02, -6.5119e-02, -3.4388e-01,
          2.4584e-01,  4.3585e-01,  2.2032e-01, -2.5319e-02,  3.2841e-01,
          7.9404e-02, -4.5297e-02,  1.3635e-01, -5.2890e-01, -2.0794e-01,
          2.5024e-01, -3.1030e-01,  1.8552e-02, -3.2511e-01, -8.5269e-02,
          8.4016e-02,  1.0609e-01, -4.8365e-02, -4.3351e-03, -2.4898e-01,
          3.8349e-02,  1.5115e-01,  2.2323e-02, -8.1496e-02, -1.3640e-01,
         -1.1187e-01,  6.24

## 예제 10.2 코드로 살펴보는 최대 모드

In [29]:
def max_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    token_embeddings[input_mask_expanded == 0] = -1e9
    return torch.max(token_embeddings, 1)[0]

## 예제 10.2 한국어 문장 임베딩 모델로 입력 문장 사이의 유사도 계산

In [ ]:
from sentence_transformers import SentenceTransformer, util

# 1. 한국어 SBERT 모델 로드 (KLUE 기반, 한국어에 특화된 사전학습 모델)
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# 2. 문장들을 SBERT로 인코딩 → 각 문장이 768차원 벡터로 변환됨
embs = model.encode(['잠이 안 옵니다',
                     '졸음이 옵니다',
                     '기차가 옵니다'])

# 3. 문장 간 코사인 유사도 계산 
cos_scores = util.cos_sim(embs, embs)

print(cos_scores)

# 출력 예시:
# tensor([[1.0000, 0.6410, 0.1887],
#         [0.6410, 1.0000, 0.2730],
#         [0.1887, 0.2730, 1.0000]])


tensor([[1.0000, 0.6410, 0.1887],
        [0.6410, 1.0000, 0.2730],
        [0.1887, 0.2730, 1.0000]])


## 예제 10.2 CLIP 모델을 활용한 이미지와 텍스트 임베딩 유사도 계산

In [ ]:
# CLIP (Contrastive Language–Image Pretraining) 모델을 사용하여
# 이미지와 텍스트 간의 의미적 유사도를 계산

from PIL import Image 
from sentence_transformers import SentenceTransformer, util

# 1. CLIP 모델 로드 
model = SentenceTransformer('clip-ViT-B-32')

# 2. 이미지 2장 로드하고 임베딩 계산 (각 이미지 → 벡터로 변환)
img_embs = model.encode([Image.open('dog.jpg'), Image.open('cat.jpg')])

# 3. 텍스트 2개 임베딩 계산 (자연어 설명 → 벡터로 변환)
text_embs = model.encode(['A dog on grass', 'Brown cat on yellow background'])

# 4. 이미지와 텍스트 임베딩 간 코사인 유사도 계산
cos_scores = util.cos_sim(img_embs, text_embs)
print(cos_scores)

# 예시 출력:
# tensor([[0.2771, 0.1509],
#         [0.2071, 0.3180]])


tensor([[0.2771, 0.1509],
        [0.2071, 0.3180]])


## 예제 10.3 실습에 사용할 모델과 데이터셋 불러오기

In [72]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

# 한국어 QA 데이터셋(KLUE MRC), train split만 사용
klue_mrc_dataset = load_dataset('klue', 'mrc', split='train')

# 2. 한국어 SBERT 모델 로드 (KLUE 기반으로 학습된 모델)
sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

## 예제 10.3 실습 데이터에서 1,000개만 선택하고 문장 임베딩으로 변환

In [73]:
klue_mrc_dataset = klue_mrc_dataset.train_test_split(train_size=1000, shuffle=False)['train']
embeddings = sentence_model.encode(klue_mrc_dataset['context'])
embeddings.shape
# 출력 결과
# (1000, 768)

(1000, 768)

## 예제 10.3 KNN 검색 인덱스를 생성하고 문장 임베딩 저장

In [ ]:
# Facebook AI의 FAISS 라이브러리를 사용하여, 문장 임베딩 벡터에 대한 빠른 유사도 검색 인덱스를 구축

import faiss

# 인덱스 만들기 (L2 거리 기반의 평면 인덱스)
index = faiss.IndexFlatL2(embeddings.shape[1])

# 인덱스에 임베딩 추가
index.add(embeddings) 
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000013CD8DFEA00> >

## 예제 10.3 의미 검색의 장점

In [78]:
query = "이번 연도에는 언제 비가 많이 올까?"

# 1. 질문 문장을 SBERT로 벡터화함
query_embedding = sentence_model.encode([query])

# 2. FAISS 인덱스에서 가장 유사한 3개의 context를 검색
# distances: 각 검색 결과의 거리 (작을수록 유사함)
# indices: 유사한 context의 인덱스들
distances, indices = index.search(query_embedding, 3)

# 3. 상위 3개 검색 결과의 context 일부 출력
for idx in indices[0]:
    print(klue_mrc_dataset['context'][idx][:50])


# 출력 결과
# 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은   (정답)
# 연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그 (오답)
# 연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그 (오답)

올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 
연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그
연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그


In [79]:
for idx in indices[0]:
    print("[문맥] ", klue_mrc_dataset['context'][idx][:50])
    print("[질문] ", klue_mrc_dataset['question'][idx])
    print("[정답] ", klue_mrc_dataset['answers'][idx]['text'])
    print()


[문맥]  올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 
[질문]  북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?
[정답]  ['한 달가량', '한 달']

[문맥]  연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그
[질문]  오리 너구리의 신체 부위 중 크게 발달한 것은?
[정답]  ['눈']

[문맥]  연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그
[질문]  오리너구리와 눈 구조가 가장 비슷한 어류는 무엇인가요?
[정답]  ['태평양먹장어(Eptatretus stoutii)', '태평양먹장어', 'Eptatretus stoutii']



## 예제 10.3 의미 검색의 한계

In [ ]:
# 1. KLUE MRC 데이터셋의 3번째 질문 가져오기
query = klue_mrc_dataset[3]['question']
# 예: "로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?"

# 2. 질문 문장을 SBERT 모델로 임베딩
query_embedding = sentence_model.encode([query])

# 3. FAISS 인덱스를 사용하여 질문 벡터와 가장 유사한 문맥(context) 검색
distances, indices = index.search(query_embedding, 3)

# 4. 유사한 context 3개의 처음 50자만 출력
for idx in indices[0]:
    print(klue_mrc_dataset['context'][idx][:50])


# 출력 결과
# 태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 (오답)
# 태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 (오답)
# 미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스 (정답)

태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 
태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 
미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스


In [81]:
print(f"[질문] {query}\n")
for idx in indices[0]:
    print("[문맥 요약] ", klue_mrc_dataset['context'][idx][:100])
    print("[전체 문맥] ", klue_mrc_dataset['context'][idx])
    print("[정답]     ", klue_mrc_dataset['answers'][idx]['text'])
    print("="*50)


[질문] 로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?

[문맥 요약]  태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 하는 미 육군 주체의 연합군 남서 태평양 방면군은 1944년 후반 마침내 필리핀을 진공하기
[전체 문맥]  태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 하는 미 육군 주체의 연합군 남서 태평양 방면군은 1944년 후반 마침내 필리핀을 진공하기로 결정했다. 그 첫 단계로 필리핀 방면의 전략 거점의 확보가 필요하였으며, 뉴기니 서쪽에 위치한 말루쿠 제도의 모로타이 섬을 공격 목표로 정했다. 또한 동시에 팔라우 제도의 펠렐리우 섬과 앙가우르 섬에도 미국 해군 주도의 연합국 중부 태평양 방면군이 공략을 맡았다.(이때의 전략 결정의 경위에 대해서는 필리핀 전투 (1944 - 1945)#미국을 참조.)

한편, 1942년에 네덜란드령 동인도의 일부였던 모로타이 섬을 점령한 일본군은 이후 수비 부대를 증강배치하지 않았다. 1944년 말루쿠 제도 방면의 방비 강화를 도모하고자 파견된 제32사단은 평야가 많은 비행장 건설에 적합한 주변의 할마헤라 섬을 방어의 중심으로 여겼다. 따라서 모로타이 섬에는 제32사단의 2개 대대가 비행장 건설을 추진했지만, 배수가 좋지 않아 건설을 포기했다. 이 2개 대대가 할마헤라 섬으로 철수한 이후에는 카와시마 타케노부(川島威伸) 중위를 지휘관으로 하는 제2유격대 소속의 2개 중대(주로 다카사고의용대)만 배치되어 있었다.

연합군이 상륙했을 때, 섬에는 9000명의 현지인이 살고 있었다. 도민에 대한 선무공작을 수행하기 위해 연합군의 상륙 부대에는 네덜란드 군 민정반이 추가 되었다.
[정답]      ['네덜란드']
[문맥 요약]  태평양 전쟁 중 뉴기니 방면에서 진공 작전을 실시해 온 더글러스 맥아더 장군을 사령관으로 하는 미 육군 주체의 연합군 남서 태평양 방면군은 1944년 후반 마침내 필리핀을 진공하기
[전

## 예제 10.3 라마인덱스에서 Sentence-Transformers 임베딩 모델 활용

In [ ]:
# llama_index의 핵심 클래스 임포트
from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.core import Document

# HuggingFace의 SBERT 임베딩 모델 로드
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# 1. 한국어 SBERT 모델을 임베딩으로 설정
embed_model = HuggingFaceEmbedding(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")

# 2. llama_index에서 사용할 서비스 컨텍스트 생성
# LLM은 생략하고 (None), 임베딩 모델만 지정
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)

# 3. KLUE MRC 데이터셋에서 context 텍스트 100개 가져오기
text_list = klue_mrc_dataset[:100]['context']

# 4. 각 텍스트를 llama_index용 Document 객체로 변환
documents = [Document(text=t) for t in text_list]

# 5. VectorStoreIndex에 문서들을 임베딩하고 저장
index_llama = VectorStoreIndex.from_documents(
    documents,
    service_context=service_context,
)

c:\Users\me\anaconda3\envs\lang310\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_38004\745163680.py:13: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)


LLM is explicitly disabled. Using MockLLM.


In [ ]:
query_engine = index_llama.as_query_engine()
response = query_engine.query("로버트 헨리 딕은 무엇을 개발했는가?")
print(response)

Context information is below.
---------------------
《존 윅》(John Wick)은 데이비드 리치와 채드 스타헬스키가 연출한 2014년 공개된 미국의 네오 누아르 액션 스릴러 영화이다. 키아누 리브스, 미카엘 뉘크비스트, 알피 앨런, 에이드리언 팰리키, 브리짓 모이나한, 딘 윈터스, 이언 맥셰인, 존 레귀자모, 윌럼 더포등이 출연했다. 존 윅의 첫 시작 작품이며, 빈티지 자동차와 최근에 사망한 아내가 선물로 남긴 강아지를 죽인것에 대한 복수를 하고자 하는 은퇴한 청부 살인업자 존 윅 (리브스)의 대한 이야기를 다루고 있다. 리치와 스타헬스키는 영화를 함께 연출했지만, 리치는 크레딧에 오르지 않았다. 

2012년에 각본을 완료했었던 데릭 콜스테드가 각본을 썼고 선더 로드 픽쳐스를 통해 제작되었다. 선더 로드 픽쳐스 사의 배질 이와닉, 리치, 에바 롱고리아, 마이클 위더릴이 제작에 참여했다. 스타헬스키와 리치에게는 제2 제작진 감독과 스턴트맨으로서 경력을 시작한 이후 첫 감독으로서의 데뷔작이다. 그들은 매트릭스 트릴로지에서 스턴트맨으로서 리브스와 함께 작업한 바가 있다.

폴 디랙은 중력 상수가 우주의 거꾸로 나이와 부정확하게 일치하며, 중력 상수는 지금의 평형을 유지하기 위해 바뀌어야만 한다고 추측했다. 딕은 디랙의 관계가 선택 효과일 수 있다는 것을 깨달았다. 평형이 깨어졌을 다른 어떠한 시대에는 그 모순을 알아차릴 수 있는 지적인 생명채가 없을 것이다. 이것은 소위 약한 인간 중심 원리의 첫 번째 현대적 적용이다.
---------------------
Given the context information and not prior knowledge, answer the query.
Query: 로버트 헨리 딕은 무엇을 개발했는가?
Answer: 


## 예제 10.14 BM25 클래스 구현

In [38]:
import math
import numpy as np
from typing import List
from transformers import PreTrainedTokenizer
from collections import defaultdict

class BM25:
  def __init__(self, corpus:List[List[str]], tokenizer:PreTrainedTokenizer):
    self.tokenizer = tokenizer
    self.corpus = corpus
    self.tokenized_corpus = self.tokenizer(corpus, add_special_tokens=False)['input_ids']
    self.n_docs = len(self.tokenized_corpus)
    self.avg_doc_lens = sum(len(lst) for lst in self.tokenized_corpus) / len(self.tokenized_corpus)
    self.idf = self._calculate_idf()
    self.term_freqs = self._calculate_term_freqs()

  def _calculate_idf(self):
    idf = defaultdict(float)
    for doc in self.tokenized_corpus:
      for token_id in set(doc):
        idf[token_id] += 1
    for token_id, doc_frequency in idf.items():
      idf[token_id] = math.log(((self.n_docs - doc_frequency + 0.5) / (doc_frequency + 0.5)) + 1)
    return idf

  def _calculate_term_freqs(self):
    term_freqs = [defaultdict(int) for _ in range(self.n_docs)]
    for i, doc in enumerate(self.tokenized_corpus):
      for token_id in doc:
        term_freqs[i][token_id] += 1
    return term_freqs

  def get_scores(self, query:str, k1:float = 1.2, b:float=0.75):
    query = self.tokenizer([query], add_special_tokens=False)['input_ids'][0]
    scores = np.zeros(self.n_docs)
    for q in query:
      idf = self.idf[q]
      for i, term_freq in enumerate(self.term_freqs):
        q_frequency = term_freq[q]
        doc_len = len(self.tokenized_corpus[i])
        score_q = idf * (q_frequency * (k1 + 1)) / ((q_frequency) + k1 * (1 - b + b * (doc_len / self.avg_doc_lens)))
        scores[i] += score_q
    return scores

  def get_top_k(self, query:str, k:int):
    scores = self.get_scores(query)
    top_k_indices = np.argsort(scores)[-k:][::-1]
    top_k_scores = scores[top_k_indices]
    return top_k_scores, top_k_indices

## 예제 10.15 BM25 점수 계산 확인해 보기

In [39]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

bm25 = BM25(['안녕하세요', '반갑습니다', '안녕 서울'], tokenizer)
bm25.get_scores('안녕')
# array([0.44713859, 0.        , 0.52354835])

array([0.44713859, 0.        , 0.52354835])

## 예제 10.16 BM25 검색 결과의 한계

In [40]:
# BM25 검색 준비
bm25 = BM25(klue_mrc_dataset['context'], tokenizer)

query = "이번 연도에는 언제 비가 많이 올까?"
_, bm25_search_ranking = bm25.get_top_k(query, 100)

for idx in bm25_search_ranking[:3]:
  print(klue_mrc_dataset['context'][idx][:50])

# 출력 결과
# 갤럭시S5 언제 발매한다는 건지언제는 “27일 판매한다”고 했다가 “이르면 26일 판매한다 (오답)
# 인구 비율당 노벨상을 세계에서 가장 많이 받은 나라, 과학 논문을 가장 많이 쓰고 의료 특 (오답)
# 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은  (정답)

Token indices sequence length is longer than the specified maximum sequence length for this model (965 > 512). Running this sequence through the model will result in indexing errors


갤럭시S5 언제 발매한다는 건지언제는 “27일 판매한다”고 했다가 “이르면 26일 판매한다
인구 비율당 노벨상을 세계에서 가장 많이 받은 나라, 과학 논문을 가장 많이 쓰고 의료 특
올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 


## 예제 10.17 BM25 검색 결과의 장점

In [41]:
query = klue_mrc_dataset[3]['question']  # 로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
_, bm25_search_ranking = bm25.get_top_k(query, 100)

for idx in bm25_search_ranking[:3]:
  print(klue_mrc_dataset['context'][idx][:50])

# 출력 결과
# 미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스 (정답)
# ;메카동(メカドン)                                                      (오답)
# :성우 : 나라하시 미키(ならはしみき)
# 길가에 버려져 있던 낡은 느티나
# ;메카동(メカドン)                                                      (오답)
# :성우 : 나라하시 미키(ならはしみき)
# 길가에 버려져 있던 낡은 느티나

미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스
;메카동(メカドン)
:성우 : 나라하시 미키(ならはしみき)
길가에 버려져 있던 낡은 느티나
;메카동(メカドン)
:성우 : 나라하시 미키(ならはしみき)
길가에 버려져 있던 낡은 느티나


## 예제 10.18 상호 순위 조합 함수 구현

In [42]:
from collections import defaultdict

def reciprocal_rank_fusion(rankings:List[List[int]], k=5):
    rrf = defaultdict(float)
    for ranking in rankings:
        for i, doc_id in enumerate(ranking, 1):
            rrf[doc_id] += 1.0 / (k + i)
    return sorted(rrf.items(), key=lambda x: x[1], reverse=True)

## 예제 10.19 예시 데이터에 대한 상호 순위 조합 결과 확인하기

In [43]:
rankings = [[1, 4, 3, 5, 6], [2, 1, 3, 6, 4]]
reciprocal_rank_fusion(rankings)

# [(1, 0.30952380952380953),
#  (3, 0.25),
#  (4, 0.24285714285714285),
#  (6, 0.2111111111111111),
#  (2, 0.16666666666666666),
#  (5, 0.1111111111111111)]

[(1, 0.30952380952380953),
 (3, 0.25),
 (4, 0.24285714285714285),
 (6, 0.2111111111111111),
 (2, 0.16666666666666666),
 (5, 0.1111111111111111)]

## 예제 10.20 하이브리드 검색 구현하기

In [44]:
def dense_vector_search(query:str, k:int):
  query_embedding = sentence_model.encode([query])
  distances, indices = index.search(query_embedding, k)
  return distances[0], indices[0]

def hybrid_search(query, k=20):
  _, dense_search_ranking = dense_vector_search(query, 100)
  _, bm25_search_ranking = bm25.get_top_k(query, 100)

  results = reciprocal_rank_fusion([dense_search_ranking, bm25_search_ranking], k=k)
  return results

## 예제 10.21 예시 데이터에 대한 하이브리드 검색 결과 확인

In [45]:
query = "이번 연도에는 언제 비가 많이 올까?"
print("검색 쿼리 문장: ", query)
results = hybrid_search(query)
for idx, score in results[:3]:
  print(klue_mrc_dataset['context'][idx][:50])

print("=" * 80)
query = klue_mrc_dataset[3]['question'] # 로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
print("검색 쿼리 문장: ", query)

results = hybrid_search(query)
for idx, score in results[:3]:
  print(klue_mrc_dataset['context'][idx][:50])

# 출력 결과
# 검색 쿼리 문장:  이번 연도에는 언제 비가 많이 올까?
# 올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은  (정답)
# 갤럭시S5 언제 발매한다는 건지언제는 “27일 판매한다”고 했다가 “이르면 26일 판매한다  (오답)
# 연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그 (오답)
# ================================================================================
# 검색 쿼리 문장:  로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
# 미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스 (정답)
# 1950년대 말 매사추세츠 공과대학교의 동아리 테크모델철도클럽에서 ‘해커’라는 용어가 처음 (오답)
# 1950년대 말 매사추세츠 공과대학교의 동아리 테크모델철도클럽에서 ‘해커’라는 용어가 처음 (오답)

검색 쿼리 문장:  이번 연도에는 언제 비가 많이 올까?
올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 
갤럭시S5 언제 발매한다는 건지언제는 “27일 판매한다”고 했다가 “이르면 26일 판매한다
연구 결과에 따르면, 오리너구리의 눈은 대부분의 포유류보다는 어류인 칠성장어나 먹장어, 그
검색 쿼리 문장:  로버트 헨리 딕이 1946년에 매사추세츠 연구소에서 개발한 것은 무엇인가?
미국 세인트루이스에서 태어났고, 프린스턴 대학교에서 학사 학위를 마치고 1939년에 로체스
1950년대 말 매사추세츠 공과대학교의 동아리 테크모델철도클럽에서 ‘해커’라는 용어가 처음
1950년대 말 매사추세츠 공과대학교의 동아리 테크모델철도클럽에서 ‘해커’라는 용어가 처음
